In [1]:
# importing libraries

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets

In [2]:
display(HTML("<div style = 'background-color: whitesnow; padding: 30px;text-align: center; '>" +
             "<span style='color: mediumslateblue; font-size:50px; font-weight:bold;text-decoration:underline;font-family:Helvetica;'>"  + "Covid-19 DASHBOARD" +"</span>" +
             "</div>")
       )

In [3]:
# Load Datasets
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [4]:
#Data Cleaning
death_df.columns = map(str.lower, death_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)
country_df['Confirmed'] = country_df['Confirmed'].astype(int)

In [5]:
country_df = country_df.rename(columns = {'Country_Region' : 'Country'})
confirmed_df = confirmed_df.rename(columns = {'province/state' : 'state', 'country/region' : 'country'})
recovered_df = recovered_df.rename(columns = {'province/state' : 'state', 'country/region' : 'country'})
death_df = death_df.rename(columns = {'province/state' : 'state', 'country/region' : 'country'})

sort_country_df = country_df.sort_values('Confirmed', ascending = False)



In [6]:
display(HTML("<div style = 'background-color: whitesnow; padding: 30px; '>" +
             "<span style='color: dodgerblue; font-size:30px;font-weight:bold;text-decoration:underline;font-family:helvetica;'>"  + "Covid-19 LIVE UPDATE" +"</span>" +
             "</div>")
       )

In [7]:
# Live Data
confirmed_total = int(country_df['Confirmed'].sum())
deaths_total = int(country_df['Deaths'].sum())
recovered_total = int(country_df['Recovered'].sum())
active_total = int(country_df['Active'].sum())

display(HTML("<div style = 'background-color: whitesnow; padding: 30px; text-align: center; '>" +
             "<span style='color: cornflowerblue; font-size:25px;'> Confirmed: "  + str(confirmed_total) +"</span>" +
             "<span style='color: orangered; font-size:25px;margin-left:20px;'> Deaths: " + str(deaths_total) + "</span>"+
             "<span style='color: lightgreen; font-size:25px; margin-left:20px;'> Recovered: " + str(recovered_total) + "</span>"+
             "</div>")
       )

In [8]:
display(HTML("<div style = 'background-color: whitesnow; padding: 30px; '>" +
             "<span style='color: darkturquoise; font-size:30px;font-weight:bold;text-decoration:underline;font-family:helvetica;'>"  + "Covid-19 TOP n COUNTRY INFO" +"</span>" +
             "</div>")
       )

In [9]:
# Top n Countries
fig = go.FigureWidget( layout=go.Layout() )

re_country_df = country_df.drop(['Lat', 'Long_','People_Tested', 'People_Hospitalized', 'UID', 'ISO3' ], axis = 1)
re_country_df['Recovered'] = re_country_df['Recovered'].apply(np.floor)
def color(x):
    a = 'background-color: mediumslateblue'
    b = 'background-color: orangered'
    c = 'background-color: springgreen'
    
    temp_df = pd.DataFrame('', index = x.index, columns = x.columns)
    
    temp_df.iloc[:,2] = a
    temp_df.iloc[:,3] = b
    temp_df.iloc[:,4] = c
    
    return temp_df
def show_latest_cases(Top_n):
    Top_n = int(Top_n)
    re_country_df['Recovered'] = re_country_df['Recovered'].apply(np.floor)
    return re_country_df.sort_values('Confirmed', ascending= False).head(Top_n).style.apply(color, axis=None)

interact(show_latest_cases, Top_n='10')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)
    

interactive(children=(Text(value='10', description='Top_n'), Output()), _dom_classes=('widget-interact',))

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

In [10]:
display(HTML("<div style = 'background-color: whitesnow; padding: 30px '>" +
             "<span style='color: crimson; font-size:30px;font-weight:bold;text-decoration:underline;font-family: helvetica;'>"  + "Covid-19 TOP AFFECTED COUNTRIES" +"</span>" +
             "</div>")
       )

In [11]:
# Bubble Chart
def bubble_chart(n):
    fig = px.scatter(sort_country_df.head(n), x="Country", y="Confirmed", size="Confirmed", color="Country",
               hover_name="Country", size_max=60)
    fig.update_layout(
    title=str(n) +" Worst Hit Countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 700
    )
    fig.show();


interact(bubble_chart, n=widgets.IntSlider(min=1, max=30, step=1, value=10))


interactive(children=(IntSlider(value=10, description='n', max=30, min=1), Output()), _dom_classes=('widget-in…

<function __main__.bubble_chart(n)>

In [12]:
display(HTML("<div style = 'background-color: whitesnow; padding: 30px '>" +
             "<span style='color: hotpink; font-size:30px;font-weight:bold;text-decoration:underline;font-family:helvetica;'>"  + "Covid-19 DATA BY COUNTRIES" +"</span>" +
             "</div>")
       )

In [13]:
# Data By Countries
import plotly.graph_objects as go

def plot_for_country(country):
    title = 'Covid 19'
    labels = ['confirmed', 'deaths', 'recovered']
    colors = ['cornflowerblue', 'orangered', 'lightgreen']

    mode_size = [6, 6, 8]
    line_size = [4, 4, 6]
    
    df_list = [confirmed_df, death_df, recovered_df]
    fig = go.Figure()
    
    for i,df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 5:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,5:]), axis = 0)
            
        else:
            x_data = np.array(list(df.iloc[:, 5:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,5:]), axis = 0)
            
        fig.add_trace(go.Scatter(
            x=x_data,
            y=y_data,
            name = labels[i],
            mode='lines+markers',
            line = dict(color = colors[i], width = line_size[i]),
            connectgaps = True,
            text = 'Total' + str(labels[i]) + ": " + str(y_data[-1])
        ))
        
        fig.update()
        
    fig.show()
    
interact(plot_for_country, country = country_df['Country'])

    

interactive(children=(Dropdown(description='country', options=('Australia', 'Austria', 'Canada', 'China', 'Den…

<function __main__.plot_for_country(country)>

In [14]:
display(HTML("<div style = 'background-color: whitesnow; padding: 30px '>" +
             "<span style='color: coral; font-size:30px;font-weight:bold;text-decoration:underline;font-family:helvetica;'>"  + "Covid-19 WORLD MAP" +"</span>" +
             "</div>")
       )

In [15]:
# World Map
import folium

world_map = folium.Map(location =[11,0], tiles = "cartodbpositron", zoom_start = 2, max_zoom = 6, min_zoom = 2 )

for i in range(len(confirmed_df)):
    folium.Circle(
    location = [confirmed_df.iloc[i]['lat'],confirmed_df.iloc[i]['long']],
    fill = True,
    radius = (int((np.log(confirmed_df.iloc[i, -1] + 1.00001))) + 0.2) * 50000 ,
    fill_color = 'cornflowerblue',
    color = 'cornflowerblue',
    tooltip = "<div style=' background-color: salmon; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                    "<hr style='margin: 10px;color: salmon;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
    ).add_to(world_map)
        
world_map


Author : Ehsanul Haque Ayon <br>
Fb ID : https://www.facebook.com/ehsanulhaque.ayon.5 <br>
LinkedIn ID: https://www.linkedin.com/in/ehsanul-haque-ayon-31885b160/ <br>
Github Repo: https://github.com/ehsanulayon/heroku-app <br>
Data : https://github.com/CSSEGISandData/COVID-19